In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
from IPython.core.display import display, HTML
import re
import dateutil.parser
from sklearn.linear_model import LinearRegression
import seaborn as sns
from datetime import date 

In [2]:
date.today().year

2022

### The goal of this code is to gather player statistics from basketball-reference.com and gather salary/contract information from ESPN.com and combine it into a dataframe. 

In [3]:
'''create a way to interate through all team page urls from basketball reference from 
2014-2022 I will need to scrape'''

#identify all the team abrevations used on the site
nba_teams = ['PHI','BRK','MIL','NYK','ATL','MIA','BOS','WAS','IND','CHO','CHI','TOR','CLE','ORL','DET',
            'UTA','PHO','DEN','LAC','DAL','POR','LAL','MEM','GSW','SAS','NOP','SAC','MIN','OKC','HOU']

#determine how many years you want to scrape
num_of_years = 8

num_list = list(range(0,num_of_years+1))

years = []
for num in num_list:
    new_year = date.today().year - num 
    new_year = str(new_year)
    years.append(new_year)

#url base url (when abbreviation is added takes you to the team page)
base_url = 'https://www.basketball-reference.com/teams/'




In [4]:
#create a list of all links to team pages

link_list= []
for team in nba_teams:
    for year in years:
        new_url = base_url + team +"/"+ year +".html"
        link_list.append(new_url)

In [5]:
'''Now that we have navigated through the page we can create a function to gather roster data from every page
   and store it in a dataframe'''


def get_roster_data(url):
    response = requests.get(url)
    web_page = response.text
    soup = BeautifulSoup(web_page,'lxml')
    try: 
        roster_results = soup.find('div', id = 'div_roster').find_all('tr')
    except AttributeError:
        return pd.DataFrame()
    roster_dict={}
    for player in roster_results[1:]:
        roster_data = player.find_all('td')
        roster_player = roster_data[0].text
        roster_dict[roster_player]=[i.text for i in roster_data]
    roster_df = pd.DataFrame(roster_dict).T
    roster_df.columns = ['PLAYER','POSITION','HEIGHT','WEIGHT','DOB','COUNTRY','EXPERIENCE','COLLEGE']
    current_year = soup.find('div', id = 'meta').find_all('span')[0].text
    roster_df['YEAR']= current_year[0:2] + current_year[-2:]
    roster_df['TEAM']=url[-13:-10]
    return roster_df


    

In [6]:
#Use get_roster_data function to gather data from every team link

roster_df_list = []
for link in link_list:
    new_df = get_roster_data(link)
    roster_df_list.append(new_df)
roster_df_complete = pd.concat(roster_df_list)

roster_df_complete


,PLAYER,POSITION,HEIGHT,WEIGHT,DOB,COUNTRY,EXPERIENCE,COLLEGE,YEAR,TEAM
Charles Bassey,Charles Bassey,C,6-11,235,"October 28, 2000",ng,R,Western Kentucky,2022,PHI
Charlie Brown Jr.,Charlie Brown Jr.,SG,6-6,199,"February 2, 1997",us,2,Saint Joseph's,2022,PHI
Willie Cauley-Stein,Willie Cauley-Stein,C,7-0,240,"August 18, 1993",us,6,Kentucky,2022,PHI
Seth Curry,Seth Curry,SG,6-2,185,"August 23, 1990",us,7,"Liberty, Duke",2022,PHI
Andre Drummond,Andre Drummond,C,6-10,279,"August 10, 1993",us,9,UConn,2022,PHI
...,...,...,...,...,...,...,...,...,...,...
Jeremy Lin,Jeremy Lin,PG,6-3,200,"August 23, 1988",us,3,Harvard University,2014,HOU
Donatas Motiejūnas,Donatas Motiejūnas,PF,7-0,222,"September 20, 1990",lt,1,,2014,HOU
Chandler Parsons,Chandler Parsons,SF,6-9,230,"October 25, 1988",us,2,Florida,2014,HOU
Josh Powell,Josh Powell,PF,6-9,225,"January 25, 1983",us,6,NC State,2014,HOU


In [7]:
#create function to collect per game data


def get_pergame_data(url):
    response = requests.get(url)
    web_page = response.text
    soup = BeautifulSoup(web_page,'lxml')
    try:
        pergame_results = soup.find('div', id = 'div_per_game').find_all('tr')
    except AttributeError:
        return pd.DataFrame()
    pergame_dict={}
    for player in pergame_results[1:]:
        pergame_data = player.find_all('td')
        pergame_player = pergame_data[0].text
        pergame_dict[pergame_player]=[i.text for i in pergame_data]
    pergame_df = pd.DataFrame(pergame_dict).T
    pergame_df.columns = ['PLAYER','AGE','GAMES','GAMES_STARTED','MINUTES_PLAYED','FIELD_GOALS','FIELD_GOALS_ATT',
                      'FIELD_GOAL_PCT','3PT_M','3PTS_A','3PT_PCT','2PT_M','2PT_A','2PT_PCT','EFG_PCT','FT_M',
                     'FT_A','FT_PCT','OFF_REB','DEF_REB','TOT_REB','ASSISTS','STEALS','BLOCKS','TO','PF','POINTS']
    current_year = soup.find('div', id = 'meta').find_all('span')[0].text
    pergame_df['YEAR']= current_year[0:2] + current_year[-2:]
                        
    pergame_df['TEAM']=url[-13:-10]
    return pergame_df

In [8]:
#get pergame data for team link

pergame_df_list = []
for link in link_list:
    new_pergame_df = get_pergame_data(link)
    pergame_df_list.append(new_pergame_df)
pergame_df_list
pergame_df_complete = pd.concat(pergame_df_list)

pergame_df_complete

,PLAYER,AGE,GAMES,GAMES_STARTED,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOALS_ATT,FIELD_GOAL_PCT,3PT_M,3PTS_A,...,DEF_REB,TOT_REB,ASSISTS,STEALS,BLOCKS,TO,PF,POINTS,YEAR,TEAM
James Harden,James Harden,32,21,21,37.7,5.5,13.6,.402,2.2,6.7,...,6.5,7.1,10.5,1.2,0.2,3.4,2.3,21.0,2022,PHI
Tyrese Maxey,Tyrese Maxey,21,75,74,35.3,6.4,13.3,.485,1.8,4.1,...,2.9,3.2,4.3,0.7,0.4,1.2,2.1,17.5,2022,PHI
Seth Curry,Seth Curry,31,45,45,34.8,5.7,11.7,.485,2.2,5.6,...,3.0,3.4,4.0,0.8,0.2,1.9,2.2,15.0,2022,PHI
Tobias Harris,Tobias Harris,29,73,73,34.8,6.8,14.0,.482,1.4,3.8,...,5.7,6.8,3.5,0.6,0.6,1.6,2.2,17.2,2022,PHI
Joel Embiid,Joel Embiid,27,68,68,33.8,9.8,19.6,.499,1.4,3.7,...,9.6,11.7,4.2,1.1,1.5,3.1,2.7,30.6,2022,PHI
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Troy Daniels,Troy Daniels,22,5,1,15.0,3.0,6.2,.484,2.4,5.0,...,0.8,0.8,1.0,0.0,0.0,0.6,1.2,8.4,2014,HOU
Isaiah Canaan,Isaiah Canaan,22,22,0,11.5,1.4,4.0,.356,0.8,2.5,...,0.8,1.1,1.0,0.4,0.2,1.0,1.1,4.6,2014,HOU
Greg Smith,Greg Smith,23,11,0,9.1,1.6,2.5,.643,0.0,0.0,...,1.5,2.5,0.0,0.1,0.2,0.5,1.5,3.5,2014,HOU
Ronnie Brewer,Ronnie Brewer,28,23,3,6.9,0.1,0.7,.200,0.0,0.3,...,0.5,0.6,0.4,0.3,0.0,0.1,0.2,0.3,2014,HOU


In [9]:
#we only want the position and experience from the roster table - drop other columns

roster_df_complete = roster_df_complete.drop(['HEIGHT','WEIGHT','DOB','COUNTRY','COLLEGE'], axis = 1)
roster_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5338 entries, Charles Bassey to Greg Smith
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   PLAYER      5338 non-null   object
 1   POSITION    5338 non-null   object
 2   EXPERIENCE  5338 non-null   object
 3   YEAR        5338 non-null   object
 4   TEAM        5338 non-null   object
dtypes: object(5)
memory usage: 250.2+ KB


In [10]:
#drop_duplicates

pergame_df_complete.drop_duplicates(inplace=True)


In [11]:
#create final df that combines list of players and their per-game stats 

final_df = pd.merge(pergame_df_complete,roster_df_complete, on=['PLAYER','YEAR','TEAM'],how = 'outer')

In [12]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5338 entries, 0 to 5337
Data columns (total 31 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   PLAYER           5338 non-null   object
 1   AGE              5333 non-null   object
 2   GAMES            5333 non-null   object
 3   GAMES_STARTED    5333 non-null   object
 4   MINUTES_PLAYED   5333 non-null   object
 5   FIELD_GOALS      5333 non-null   object
 6   FIELD_GOALS_ATT  5333 non-null   object
 7   FIELD_GOAL_PCT   5333 non-null   object
 8   3PT_M            5333 non-null   object
 9   3PTS_A           5333 non-null   object
 10  3PT_PCT          5333 non-null   object
 11  2PT_M            5333 non-null   object
 12  2PT_A            5333 non-null   object
 13  2PT_PCT          5333 non-null   object
 14  EFG_PCT          5333 non-null   object
 15  FT_M             5333 non-null   object
 16  FT_A             5333 non-null   object
 17  FT_PCT           5333 non-null   

In [13]:
#dropping the 5 players with NaN because there is not any data for any of them

final_df = final_df[final_df['AGE'].notna()]

In [14]:
#dropping these columns because they are not relevant or captured by another column 

final_df = final_df.drop(['AGE','GAMES_STARTED','FIELD_GOALS_ATT',
                      '3PTS_A','2PT_A','FT_A'],axis=1)

In [15]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5333 entries, 0 to 5332
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   PLAYER          5333 non-null   object
 1   GAMES           5333 non-null   object
 2   MINUTES_PLAYED  5333 non-null   object
 3   FIELD_GOALS     5333 non-null   object
 4   FIELD_GOAL_PCT  5333 non-null   object
 5   3PT_M           5333 non-null   object
 6   3PT_PCT         5333 non-null   object
 7   2PT_M           5333 non-null   object
 8   2PT_PCT         5333 non-null   object
 9   EFG_PCT         5333 non-null   object
 10  FT_M            5333 non-null   object
 11  FT_PCT          5333 non-null   object
 12  OFF_REB         5333 non-null   object
 13  DEF_REB         5333 non-null   object
 14  TOT_REB         5333 non-null   object
 15  ASSISTS         5333 non-null   object
 16  STEALS          5333 non-null   object
 17  BLOCKS          5333 non-null   object
 18  TO      

In [16]:
#converting the data types from object to numeric

column_list = list(final_df.columns)
column_list = column_list[2:21]
column_list

final_df[column_list] = final_df[column_list].apply(pd.to_numeric, errors='coerce', axis=1)

In [17]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5333 entries, 0 to 5332
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          5333 non-null   object 
 1   GAMES           5333 non-null   object 
 2   MINUTES_PLAYED  5333 non-null   float64
 3   FIELD_GOALS     5333 non-null   float64
 4   FIELD_GOAL_PCT  5291 non-null   float64
 5   3PT_M           5333 non-null   float64
 6   3PT_PCT         4839 non-null   float64
 7   2PT_M           5333 non-null   float64
 8   2PT_PCT         5236 non-null   float64
 9   EFG_PCT         5291 non-null   float64
 10  FT_M            5333 non-null   float64
 11  FT_PCT          4988 non-null   float64
 12  OFF_REB         5333 non-null   float64
 13  DEF_REB         5333 non-null   float64
 14  TOT_REB         5333 non-null   float64
 15  ASSISTS         5333 non-null   float64
 16  STEALS          5333 non-null   float64
 17  BLOCKS          5333 non-null   f

In [18]:
#drop NAs for POS (all low level players with minimal playing time)

final_df.dropna(subset=['POSITION'], inplace = True)

In [19]:
#games need to be int data type

final_df['GAMES'] = final_df['GAMES'].astype('int')


#dropping rookie players since their pay is not based on performance

final_df.drop(final_df[final_df['EXPERIENCE']=='R'].index, inplace=True)

#experience also needs to be an int

final_df['EXPERIENCE'] = final_df['EXPERIENCE'].astype('int')
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394 entries, 0 to 5331
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          4394 non-null   object 
 1   GAMES           4394 non-null   int64  
 2   MINUTES_PLAYED  4394 non-null   float64
 3   FIELD_GOALS     4394 non-null   float64
 4   FIELD_GOAL_PCT  4371 non-null   float64
 5   3PT_M           4394 non-null   float64
 6   3PT_PCT         4020 non-null   float64
 7   2PT_M           4394 non-null   float64
 8   2PT_PCT         4334 non-null   float64
 9   EFG_PCT         4371 non-null   float64
 10  FT_M            4394 non-null   float64
 11  FT_PCT          4190 non-null   float64
 12  OFF_REB         4394 non-null   float64
 13  DEF_REB         4394 non-null   float64
 14  TOT_REB         4394 non-null   float64
 15  ASSISTS         4394 non-null   float64
 16  STEALS          4394 non-null   float64
 17  BLOCKS          4394 non-null   f

In [20]:
 #check NAs for fg pct. all players with only 1-2 games and little minutes. Will be dropped. 
    
final_df[final_df['FIELD_GOAL_PCT'].isna()]

,PLAYER,GAMES,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOAL_PCT,3PT_M,3PT_PCT,2PT_M,2PT_PCT,EFG_PCT,...,ASSISTS,STEALS,BLOCKS,TO,PF,POINTS,YEAR,TEAM,POSITION,EXPERIENCE
20,Willie Cauley-Stein,2,3.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.5,0.0,0.0,0.0,1.0,0.0,2022,PHI,C,6
224,Wenyen Gabriel,1,1.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2022,BRK,PF,2
316,Rashad Vaughn,1,4.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,1.0,0.0,0.0,0.0,1.0,0.0,2018,BRK,SG,2
1069,James Ennis III,3,2.3,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.3,0.0,0.0,0.0,0.3,0.0,2016,MIA,SF,1
1129,Juwan Morgan,1,4.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2022,BOS,PF,2
1133,C.J. Miles,1,2.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,2022,BOS,SG,15
1795,Jordan Bell,1,2.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.0,1.0,0.0,0.0,2.0,0.0,2022,CHI,C,4
1856,Tyler Ulis,1,1.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2019,CHI,PG,2
1942,Ronnie Brewer,1,2.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,2014,CHI,SF,7
1943,Jarvis Varnado,1,2.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.0,0.0,0.0,0.0,1.0,0.0,2014,CHI,PF,1


In [21]:
#NAs for FG, 3pt,2pt, and ft PCT represent players who did not attempt those types of shots. Will fill with 0
final_df[final_df['3PT_PCT'].isna()]

,PLAYER,GAMES,MINUTES_PLAYED,FIELD_GOALS,FIELD_GOAL_PCT,3PT_M,3PT_PCT,2PT_M,2PT_PCT,EFG_PCT,...,ASSISTS,STEALS,BLOCKS,TO,PF,POINTS,YEAR,TEAM,POSITION,EXPERIENCE
11,DeAndre Jordan,16,13.4,2.0,0.593,0.0,NaN,2.0,0.593,0.593,...,0.5,0.1,0.6,1.0,1.3,4.6,2022,PHI,C,13
20,Willie Cauley-Stein,2,3.0,0.0,NaN,0.0,NaN,0.0,NaN,NaN,...,0.5,0.0,0.0,0.0,1.0,0.0,2022,PHI,C,6
41,Gary Clark,2,6.5,0.0,0.000,0.0,NaN,0.0,0.000,0.000,...,0.5,0.5,0.0,0.0,0.5,0.0,2021,PHI,PF,2
106,Jahlil Okafor,2,12.5,2.0,0.444,0.0,NaN,2.0,0.444,0.444,...,0.5,0.0,1.0,1.5,3.5,5.0,2018,PHI,C,2
123,Jahlil Okafor,50,22.7,4.8,0.514,0.0,NaN,4.8,0.514,0.514,...,1.2,0.4,1.0,1.8,2.4,11.8,2017,PHI,C,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5265,Clint Capela,65,23.9,5.6,0.643,0.0,NaN,5.6,0.643,0.643,...,1.0,0.5,1.2,1.3,2.8,12.6,2017,HOU,C,2
5290,Chuck Hayes,2,12.0,0.5,0.500,0.0,NaN,0.5,0.500,0.500,...,1.0,0.0,0.0,1.5,3.0,2.0,2016,HOU,C,10
5321,Ömer Aşık,48,20.2,2.1,0.532,0.0,NaN,2.1,0.532,0.532,...,0.5,0.3,0.8,1.2,1.9,5.8,2014,HOU,C,3
5323,Josh Powell,1,19.0,2.0,0.333,0.0,NaN,2.0,0.333,0.333,...,0.0,0.0,1.0,1.0,1.0,4.0,2014,HOU,PF,6


In [22]:
#players who did not make attempts at 3pt fgs, 2pt fgs, and fts will be filled with 0 

final_df['3PT_PCT']= final_df['3PT_PCT'].fillna(0)
final_df['2PT_PCT']=final_df['2PT_PCT'].fillna(0)
final_df['FT_PCT']=final_df['FT_PCT'].fillna(0)
final_df['EFG_PCT']=final_df['EFG_PCT'].fillna(0)
final_df['FIELD_GOAL_PCT']=final_df['FIELD_GOAL_PCT'].fillna(0)
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394 entries, 0 to 5331
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          4394 non-null   object 
 1   GAMES           4394 non-null   int64  
 2   MINUTES_PLAYED  4394 non-null   float64
 3   FIELD_GOALS     4394 non-null   float64
 4   FIELD_GOAL_PCT  4394 non-null   float64
 5   3PT_M           4394 non-null   float64
 6   3PT_PCT         4394 non-null   float64
 7   2PT_M           4394 non-null   float64
 8   2PT_PCT         4394 non-null   float64
 9   EFG_PCT         4394 non-null   float64
 10  FT_M            4394 non-null   float64
 11  FT_PCT          4394 non-null   float64
 12  OFF_REB         4394 non-null   float64
 13  DEF_REB         4394 non-null   float64
 14  TOT_REB         4394 non-null   float64
 15  ASSISTS         4394 non-null   float64
 16  STEALS          4394 non-null   float64
 17  BLOCKS          4394 non-null   f

### Grabbing Salary from ESPN

In [23]:
#enter # of available pages for salary info

num_of_urls = 14 

pagenums = np.arange(1,num_of_urls,1)
url_list = []
for year_data in years:
    url_year_list = []
    for num in pagenums:
        url2021 = f'http://www.espn.com/nba/salaries/_/year/{year_data}/page/'+str(num)
        url_year_list.append(url2021)
    url_list.append(url_year_list)


In [24]:
#create function to gether salary data from a single page. 

def get_salary_data(url):
    response = requests.get(url)
    web_page = response.text
    soup = BeautifulSoup(web_page,'lxml')
    try:
        salary_results = soup.find('div', id='my-players-table').find_all('tr')
        salary_dict={}
        for row in salary_results[1:]:
            if len(salary_results)==1:
                continue
            salary_data = row.find_all('td')
            salary_player = salary_data[0].text
            salary_dict[salary_player]=[i.text for i in salary_data]
        if len(salary_dict) > 1:
            salary_df = pd.DataFrame(salary_dict).T
            salary_df.columns = ['RANK','PLAYER','TEAM','SALARY']
            salary_df.drop(salary_df[salary_df['RANK']=='RK'].index, inplace =True)
            if len(url) ==52:
                salary_df['YEAR']= url[-12:-8]
            else:
                salary_df['YEAR']= url[-11:-7] 
            return salary_df
    except AttributeError:
        pass
        

In [25]:
#get Salary Data from all pages for all years

salary_df_list = []
for outter_url in url_list:
    for inner_url in outter_url:
        new_salary_df = get_salary_data(inner_url)
        salary_df_list.append(new_salary_df)
salary_df_complete = pd.concat(salary_df_list)
                      
salary_df_complete.info()                        

<class 'pandas.core.frame.DataFrame'>
Index: 4557 entries, 1 to 434
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   RANK    4557 non-null   object
 1   PLAYER  4557 non-null   object
 2   TEAM    4557 non-null   object
 3   SALARY  4557 non-null   object
 4   YEAR    4557 non-null   object
dtypes: object(5)
memory usage: 213.6+ KB


In [26]:
#replace $ and , with empty string for formatting 

salary_df_complete['SALARY'] = salary_df_complete['SALARY'].str.replace('$','')
salary_df_complete['SALARY'] = salary_df_complete['SALARY'].str.replace(',','')
salary_df_complete.head()



,RANK,PLAYER,TEAM,SALARY,YEAR
1,1,"Stephen Curry, PG",Golden State Warriors,45780966,2022
2,2,"James Harden, SG",Philadelphia 76ers,44310840,2022
3,3,"John Wall, PG",LA Clippers,44310840,2022
4,4,"Russell Westbrook, PG",Los Angeles Lakers,44211146,2022
5,5,"Kevin Durant, PF",Brooklyn Nets,42018900,2022


In [27]:
#get rid of comma and position in player column for formatting

sal_name_list = []
for values in salary_df_complete['PLAYER'].values:
    new_value = values.split(',')[0]
    sal_name_list.append(new_value)

#update player name columns    
salary_df_complete['PLAYER']=sal_name_list

In [28]:
#change salary to int 

salary_df_complete['SALARY'] = salary_df_complete['SALARY'].astype('int')

In [29]:
salary_df_complete.head()

,RANK,PLAYER,TEAM,SALARY,YEAR
1,1,Stephen Curry,Golden State Warriors,45780966,2022
2,2,James Harden,Philadelphia 76ers,44310840,2022
3,3,John Wall,LA Clippers,44310840,2022
4,4,Russell Westbrook,Los Angeles Lakers,44211146,2022
5,5,Kevin Durant,Brooklyn Nets,42018900,2022


In [30]:
#merge final(stats) df with the salary df

final_df = pd.merge(final_df,salary_df_complete, on=['PLAYER','YEAR'],how = "left")

final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4394 entries, 0 to 4393
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          4394 non-null   object 
 1   GAMES           4394 non-null   int64  
 2   MINUTES_PLAYED  4394 non-null   float64
 3   FIELD_GOALS     4394 non-null   float64
 4   FIELD_GOAL_PCT  4394 non-null   float64
 5   3PT_M           4394 non-null   float64
 6   3PT_PCT         4394 non-null   float64
 7   2PT_M           4394 non-null   float64
 8   2PT_PCT         4394 non-null   float64
 9   EFG_PCT         4394 non-null   float64
 10  FT_M            4394 non-null   float64
 11  FT_PCT          4394 non-null   float64
 12  OFF_REB         4394 non-null   float64
 13  DEF_REB         4394 non-null   float64
 14  TOT_REB         4394 non-null   float64
 15  ASSISTS         4394 non-null   float64
 16  STEALS          4394 non-null   float64
 17  BLOCKS          4394 non-null   f

In [31]:
#drop players we do not have salary information on
final_df.dropna(subset=['SALARY'], inplace = True)


In [32]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3618 entries, 0 to 4393
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          3618 non-null   object 
 1   GAMES           3618 non-null   int64  
 2   MINUTES_PLAYED  3618 non-null   float64
 3   FIELD_GOALS     3618 non-null   float64
 4   FIELD_GOAL_PCT  3618 non-null   float64
 5   3PT_M           3618 non-null   float64
 6   3PT_PCT         3618 non-null   float64
 7   2PT_M           3618 non-null   float64
 8   2PT_PCT         3618 non-null   float64
 9   EFG_PCT         3618 non-null   float64
 10  FT_M            3618 non-null   float64
 11  FT_PCT          3618 non-null   float64
 12  OFF_REB         3618 non-null   float64
 13  DEF_REB         3618 non-null   float64
 14  TOT_REB         3618 non-null   float64
 15  ASSISTS         3618 non-null   float64
 16  STEALS          3618 non-null   float64
 17  BLOCKS          3618 non-null   f

### GRAB CONTRACT YEARS

In [33]:
#hardcode names of NBA teams to add to base url 

contract_team_list = ['boston-celtics','brooklyn-nets','new-york-knicks','philadelphia-76ers','toronto-raptors',
                     'chicago-bulls','cleveland-cavaliers','detroit-pistons','indiana-pacers','milwaukee-bucks',
                     'atlanta-hawks','charlotte-hornets','miami-heat','orlando-magic','washington-wizards',
                    'denver-nuggets','minnesota-timberwolves','oklahoma-city-thunder','portland-trail-blazers',
                     'utah-jazz','golden-state-warriors','los-angeles-clippers','los-angeles-lakers','phoenix-suns',
                     'sacramento-kings','dallas-mavericks','houston-rockets','memphis-grizzlies','new-orleans-pelicans',
                     'san-antonio-spurs']
contract_url_list=[]

for team in contract_team_list:
    contract_url = 'https://www.spotrac.com/nba/'+team+'/contracts/' #established a base url 
    contract_url_list.append(contract_url)

In [34]:
def get_contract_data(url):
    
#get html code from webpage
    contract_response = requests.get(url)
    contract_page = contract_response.text
    contract_soup = BeautifulSoup(contract_page,'lxml')
    
#search for the table rows
    contract_table = contract_soup.find_all('div',class_='teams')[0].find_all('tr')
    
#populate table data into dict 
    contract_dict = {}
    for row in contract_table[1:]:
        contract_data = row.find_all('td')
        contract_player = contract_data[0].text
        contract_dict[contract_player]=[i.text for i in contract_data]
        
#turn dict into a df
    contract_df = pd.DataFrame(contract_dict).T
    contract_df.columns = ['PLAYER','POSITION','AGE','EXP','CONTRACT','AVG_SAL','GUARANTEED','EXPIRES']
    
#split apart contract column to create two new columns for contract length and total    
    contract_len_list = []
    contract_tot_list = []
    for value in contract_df['CONTRACT']:
        value_list= value.split('$') 
        string = value_list[0] #only look at text before the "$" sign
        total_year = string[-5:] # last 4 elements of string are the length of contract + 'yr'
        total_year = total_year[0] #only grab the first element of string which is the number of years in contract
        total_amount = string[0:-5:] #elements of string up to the length + "yr" is the contract amount
        contract_len_list.append(total_year)
        contract_tot_list.append(total_amount)
    contract_df['CONTRACT_LEN']=contract_len_list 
    contract_df['CONTRACT_TOT']=contract_tot_list
    return contract_df


In [35]:
#combine all contract data into df

contract_df_list=[]
for url in contract_url_list:
    new_df=get_contract_data(url)
    contract_df_list.append(new_df)
contract_df_complete = pd.concat(contract_df_list)

display(contract_df_complete.head())
display(contract_df_complete.info())
    
    
    


,PLAYER,POSITION,AGE,EXP,CONTRACT,AVG_SAL,GUARANTEED,EXPIRES,CONTRACT_LEN,CONTRACT_TOT
TatumJayson Tatum,TatumJayson Tatum,PF,24,5,"1630003005 yr $163,000,300\n","$32,600,060","$163,000,300",2026,5,163000300
HorfordAl Horford,HorfordAl Horford,C,36,15,"1090000004 yr $109,000,000\n","$27,250,000","$102,000,000",2023,4,109000000
BrownJaylen Brown,BrownJaylen Brown,SG,25,6,"1063333344 yr $106,333,334\n","$26,583,334","$103,000,000",2024,4,106333334
BrogdonMalcolm Brogdon,BrogdonMalcolm Brogdon,PG,29,6,"450000002 yr $45,000,000\n","$22,500,000","$45,000,000",2025,2,45000000
SmartMarcus Smart,SmartMarcus Smart,PG,28,8,"764879964 yr $76,487,996\n","$19,121,999","$76,487,996",2026,4,76487996


<class 'pandas.core.frame.DataFrame'>
Index: 539 entries, TatumJayson Tatum  to HallJordan Hall 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PLAYER        539 non-null    object
 1   POSITION      539 non-null    object
 2   AGE           539 non-null    object
 3   EXP           539 non-null    object
 4   CONTRACT      539 non-null    object
 5   AVG_SAL       539 non-null    object
 6   GUARANTEED    539 non-null    object
 7   EXPIRES       539 non-null    object
 8   CONTRACT_LEN  539 non-null    object
 9   CONTRACT_TOT  539 non-null    object
dtypes: object(10)
memory usage: 46.3+ KB


None

In [36]:
#create function to update player name format
def clean_player_name(player_name_series):
    contract_player_list = []
    for string in player_name_series:
        string_list = string.split()
        new_str = string_list[0].replace(string_list[0],string_list[0][len(string_list[1]):])
        full_string = new_str +' '+ string_list[1]
        contract_player_list.append(full_string)
    player_name_series = contract_player_list
    return player_name_series

In [37]:
contract_df_complete['PLAYER'] = clean_player_name(contract_df_complete['PLAYER'])

In [38]:
contract_df_complete

,PLAYER,POSITION,AGE,EXP,CONTRACT,AVG_SAL,GUARANTEED,EXPIRES,CONTRACT_LEN,CONTRACT_TOT
TatumJayson Tatum,Jayson Tatum,PF,24,5,"1630003005 yr $163,000,300\n","$32,600,060","$163,000,300",2026,5,163000300
HorfordAl Horford,Al Horford,C,36,15,"1090000004 yr $109,000,000\n","$27,250,000","$102,000,000",2023,4,109000000
BrownJaylen Brown,Jaylen Brown,SG,25,6,"1063333344 yr $106,333,334\n","$26,583,334","$103,000,000",2024,4,106333334
BrogdonMalcolm Brogdon,Malcolm Brogdon,PG,29,6,"450000002 yr $45,000,000\n","$22,500,000","$45,000,000",2025,2,45000000
SmartMarcus Smart,Marcus Smart,PG,28,8,"764879964 yr $76,487,996\n","$19,121,999","$76,487,996",2026,4,76487996
...,...,...,...,...,...,...,...,...,...,...
DiengGorgui Dieng,Gorgui Dieng,C,32,9,"26416821 yr $2,641,682\n","$2,641,682","$2,641,682",2023,1,2641682
JohnsonAlize Johnson,Alize Johnson,PF,26,4,"19681751 yr $1,968,175\n","$1,968,175",-,2023,1,1968175
KuhseTommy Kuhse,Tommy Kuhse,G,24,,"10177811 yr $1,017,781\n","$1,017,781",-,2023,1,1017781
BarlowDominick Barlow,Dominick Barlow,F,19,,01 yr -\n,-,-,2023,y,01


In [39]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 539 entries, TatumJayson Tatum  to HallJordan Hall 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   PLAYER        539 non-null    object
 1   POSITION      539 non-null    object
 2   AGE           539 non-null    object
 3   EXP           539 non-null    object
 4   CONTRACT      539 non-null    object
 5   AVG_SAL       539 non-null    object
 6   GUARANTEED    539 non-null    object
 7   EXPIRES       539 non-null    object
 8   CONTRACT_LEN  539 non-null    object
 9   CONTRACT_TOT  539 non-null    object
dtypes: object(10)
memory usage: 46.3+ KB


In [40]:
contract_df_complete['EXPIRES']=contract_df_complete['EXPIRES'].replace('-',np.NaN)


contract_df_complete['EXPIRES'].values

array(['2026', '2023', '2024', '2025', '2026', '2025', '2026', '2024',
       '2023', '2024', '2025', '2024', '2023', '2023', '2023', '2023',
       '2026', '2025', '2023', '2024', '2024', '2023', '2024', '2024',
       '2025', '2025', '2024', '2024', '2023', '2023', '2023', '2023',
       '2026', '2023', '2026', '2025', '2026', '2024', '2024', '2023',
       '2024', '2025', '2024', '2025', '2024', '2023', '2023', '2023',
       '2024', '2024', '2024', '2025', '2024', '2023', '2024', '2025',
       '2024', '2023', '2024', '2023', '2024', '2023', '2023', '2023',
       nan, nan, '2023', '2024', '2024', '2025', '2024', '2025', '2025',
       '2024', '2024', '2024', '2024', '2024', '2025', '2024', '2024',
       '2023', '2023', '2023', '2023', '2023', '2023', '2027', '2023',
       '2024', '2025', '2025', '2024', '2023', '2026', '2024', '2024',
       '2024', '2023', '2023', '2023', '2023', '2023', '2023', '2023',
       '2023', '2023', '2026', '2023', '2026', '2023', '2025', '2024',
    

In [41]:
contract_df_complete[contract_df_complete['EXPIRES'].isna()].sort_values('EXP')

,PLAYER,POSITION,AGE,EXP,CONTRACT,AVG_SAL,GUARANTEED,EXPIRES,CONTRACT_LEN,CONTRACT_TOT
Brown Jr.Charlie Brown Jr.,Jr.Charlie,SG,24,3,02 yr -\n,-,-,NaN,y,02
GaffordDaniel Gafford,Daniel Gafford,PF,23,3,"61295934 yr $6,129,593\n","$1,532,398","$6,129,593",NaN,4,6129593
JohnsonKeldon Johnson,Keldon Johnson,SG,22,3,"100173854 yr $10,017,385\n","$2,504,346","$10,017,385",NaN,4,10017385
ChealeyJoe Chealey,Joe Chealey,PG,26,4,"8167810 day $81,678\n","$8,168","$81,678",NaN,,8167810
McClellanSheldon McClellan,Sheldon McClellan,SG,29,5,"9155710 day $91,557\n","$9,156","$91,557",NaN,,9155710
Cauley-SteinWillie Cauley-Stein,Willie Cauley-Stein,C,29,7,"12008310 day $120,083\n","$12,008","$120,083",NaN,,12008310
ConnaughtonPat Connaughton,Pat Connaughton,SG,29,7,"160000003 yr $16,000,000\n","$5,333,333","$16,000,000",NaN,3,16000000


In [42]:
#drop na's from expired because they are either 10 day contracts of rookie contracts

contract_df_complete.dropna(subset=['EXPIRES'], inplace = True)

In [43]:
#drop values that dont provide salary amount 
contract_df_complete[contract_df_complete['AVG_SAL']=='-']
contract_df_complete['AVG_SAL']=contract_df_complete['AVG_SAL'].replace('-',np.NaN)
contract_df_complete.dropna(subset=['AVG_SAL'], inplace = True)

In [44]:
contract_df_complete['CONTRACT_LEN']=contract_df_complete['CONTRACT_LEN'].replace(' ',np.NaN)

In [45]:
contract_df_complete['EXPIRES']=contract_df_complete['EXPIRES'].astype('float')
contract_df_complete['CONTRACT_TOT']=contract_df_complete['CONTRACT_TOT'].astype('float')


In [46]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484 entries, TatumJayson Tatum  to KuhseTommy Kuhse 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        484 non-null    object 
 1   POSITION      484 non-null    object 
 2   AGE           484 non-null    object 
 3   EXP           484 non-null    object 
 4   CONTRACT      484 non-null    object 
 5   AVG_SAL       484 non-null    object 
 6   GUARANTEED    484 non-null    object 
 7   EXPIRES       484 non-null    float64
 8   CONTRACT_LEN  484 non-null    object 
 9   CONTRACT_TOT  484 non-null    float64
dtypes: float64(2), object(8)
memory usage: 41.6+ KB


In [47]:
#drop players who signed 10-day contracts

contract_df_complete[contract_df_complete['CONTRACT_LEN']==' ']
contract_df_complete['CONTRACT_LEN']=contract_df_complete['CONTRACT_LEN'].replace(' ',np.NaN)
contract_df_complete.dropna(subset=['CONTRACT_LEN'], inplace = True)


In [48]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484 entries, TatumJayson Tatum  to KuhseTommy Kuhse 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        484 non-null    object 
 1   POSITION      484 non-null    object 
 2   AGE           484 non-null    object 
 3   EXP           484 non-null    object 
 4   CONTRACT      484 non-null    object 
 5   AVG_SAL       484 non-null    object 
 6   GUARANTEED    484 non-null    object 
 7   EXPIRES       484 non-null    float64
 8   CONTRACT_LEN  484 non-null    object 
 9   CONTRACT_TOT  484 non-null    float64
dtypes: float64(2), object(8)
memory usage: 41.6+ KB


In [49]:
contract_df_complete= contract_df_complete[contract_df_complete['CONTRACT_LEN']!= ' ']

In [50]:
#function change data type for 3 columns
update_series_type_list = [contract_df_complete['CONTRACT_LEN'],
                           contract_df_complete['EXPIRES']]
def change_to_float(panda_series):
    panda_series =panda_series.astype('float')
    return panda_series

In [51]:
#formatting


for item in update_series_type_list:
    change_to_float(item)

In [52]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484 entries, TatumJayson Tatum  to KuhseTommy Kuhse 
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        484 non-null    object 
 1   POSITION      484 non-null    object 
 2   AGE           484 non-null    object 
 3   EXP           484 non-null    object 
 4   CONTRACT      484 non-null    object 
 5   AVG_SAL       484 non-null    object 
 6   GUARANTEED    484 non-null    object 
 7   EXPIRES       484 non-null    float64
 8   CONTRACT_LEN  484 non-null    object 
 9   CONTRACT_TOT  484 non-null    float64
dtypes: float64(2), object(8)
memory usage: 41.6+ KB


In [53]:
#formatting

contract_df_complete['CONTRACT_LEN'] = contract_df_complete['CONTRACT_LEN'].astype("float")

In [54]:
#create a column to find out the year the contract was signed so we can only take stats from years prior 
contract_df_complete["YEAR_SIGNED"]= contract_df_complete['EXPIRES']-contract_df_complete['CONTRACT_LEN']

In [55]:
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484 entries, TatumJayson Tatum  to KuhseTommy Kuhse 
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        484 non-null    object 
 1   POSITION      484 non-null    object 
 2   AGE           484 non-null    object 
 3   EXP           484 non-null    object 
 4   CONTRACT      484 non-null    object 
 5   AVG_SAL       484 non-null    object 
 6   GUARANTEED    484 non-null    object 
 7   EXPIRES       484 non-null    float64
 8   CONTRACT_LEN  484 non-null    float64
 9   CONTRACT_TOT  484 non-null    float64
 10  YEAR_SIGNED   484 non-null    float64
dtypes: float64(4), object(7)
memory usage: 45.4+ KB


In [56]:
#drop columns since they will not be used

contract_df_complete = contract_df_complete.drop(['POSITION','AGE','EXP','CONTRACT','AVG_SAL','GUARANTEED'],axis=1)
contract_df_complete.info()

<class 'pandas.core.frame.DataFrame'>
Index: 484 entries, TatumJayson Tatum  to KuhseTommy Kuhse 
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PLAYER        484 non-null    object 
 1   EXPIRES       484 non-null    float64
 2   CONTRACT_LEN  484 non-null    float64
 3   CONTRACT_TOT  484 non-null    float64
 4   YEAR_SIGNED   484 non-null    float64
dtypes: float64(4), object(1)
memory usage: 22.7+ KB


In [57]:
#update formatting of player column to all lower case 

contract_df_complete['PLAYER']=contract_df_complete['PLAYER'].str.lower()
final_df['PLAYER']=final_df['PLAYER'].str.lower()

In [58]:
#merge previous df with the new df with contract info

final_df_3 = pd.merge(final_df,contract_df_complete, on=['PLAYER'],how = 'inner')
final_df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1522 entries, 0 to 1521
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          1522 non-null   object 
 1   GAMES           1522 non-null   int64  
 2   MINUTES_PLAYED  1522 non-null   float64
 3   FIELD_GOALS     1522 non-null   float64
 4   FIELD_GOAL_PCT  1522 non-null   float64
 5   3PT_M           1522 non-null   float64
 6   3PT_PCT         1522 non-null   float64
 7   2PT_M           1522 non-null   float64
 8   2PT_PCT         1522 non-null   float64
 9   EFG_PCT         1522 non-null   float64
 10  FT_M            1522 non-null   float64
 11  FT_PCT          1522 non-null   float64
 12  OFF_REB         1522 non-null   float64
 13  DEF_REB         1522 non-null   float64
 14  TOT_REB         1522 non-null   float64
 15  ASSISTS         1522 non-null   float64
 16  STEALS          1522 non-null   float64
 17  BLOCKS          1522 non-null   f

In [59]:
#update year data types
final_df_3['YEAR'] = final_df_3['YEAR'].astype('int')
final_df_3['EXPERIENCE']=final_df_3['EXPERIENCE'].astype('float')

In [60]:
#Only take into account stats from years that took place BEFORE (or less than) their current contract year
final_df_3 = final_df_3[final_df_3['YEAR']<final_df_3['YEAR_SIGNED']]

In [61]:
final_df_3 = final_df_3.drop_duplicates()


In [62]:
final_df_3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 962 entries, 2 to 1513
Data columns (total 32 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PLAYER          962 non-null    object 
 1   GAMES           962 non-null    int64  
 2   MINUTES_PLAYED  962 non-null    float64
 3   FIELD_GOALS     962 non-null    float64
 4   FIELD_GOAL_PCT  962 non-null    float64
 5   3PT_M           962 non-null    float64
 6   3PT_PCT         962 non-null    float64
 7   2PT_M           962 non-null    float64
 8   2PT_PCT         962 non-null    float64
 9   EFG_PCT         962 non-null    float64
 10  FT_M            962 non-null    float64
 11  FT_PCT          962 non-null    float64
 12  OFF_REB         962 non-null    float64
 13  DEF_REB         962 non-null    float64
 14  TOT_REB         962 non-null    float64
 15  ASSISTS         962 non-null    float64
 16  STEALS          962 non-null    float64
 17  BLOCKS          962 non-null    fl

In [63]:
final_df_3.to_csv('linreg_final_df.csv',index=False)